In [1]:

import pandas as pd
import numpy as np

In [2]:
## cleaning tool


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [3]:
##note KNN or other clusters might be helpful group the teams in smart way ... but not now.
#models


from sklearn.preprocessing import StandardScaler, MinMaxScaler 
std_scal = StandardScaler()
mm_scal = MinMaxScaler()


##no tuning 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


from sklearn.model_selection import train_test_split



##regression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

#classifiers (non-tree)
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDRegressor, SGDClassifier
from sklearn.svm import SVC


#tree-based classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor

##regression models
lr = Ridge(alpha=0.001) 
rfr = RandomForestRegressor(max_depth=3, random_state=0)
xgbr = XGBRegressor()

##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0)
svc = SVC()

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=3, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()



In [4]:
y_cols = ['Unnamed: 0', 'game_id', 'mp_date', 'season', 'home_team', 'away_team',
       'home_odds', 'away_odds', 'home_goals', 'away_goals', 'goal_diff_target', 'home_win',
       'settled_in', ]

x_cols = ['CF%', 'CSh%', 'CSv%', 'FF%', 'FSh%', 'FSv%', 'GDiff',
       'GF%', 'PDO', 'PENDiff', 'SF%', 'SDiff', 'Sh%', 'Sv']
columns_to_scale = ['CF%', 'CSh%', 'CSv%', 'FF%', 'FSh%', 'FSv%', 'GDiff',
       'GF%', 'PDO', 'PENDiff', 'SF%', 'SDiff', 'Sh%', 'Sv']  ##same

In [5]:
X = pd.read_csv('/Users/joejohns/data_bootcamp/GitHub/final_project_nhl_prediction/Data/Shaped_Data/data_LJ.csv')

X['season'] = X['season'].apply(int)
X['game_id'] = X['game_id'].apply(int)
X['mp_date'] = X['mp_date'].apply(int)
X['goal_diff_target'] = X['home_goals'] - X['away_goals']
X.loc[X['home_odds'] == X['away_odds'],  ['away_odds']]=110  #(!)
X.shape

(10385, 27)

In [6]:
X.head(10)

,Unnamed: 0,game_id,mp_date,season,home_team,away_team,home_odds,away_odds,home_goals,away_goals,...,FSv%,GDiff,GF%,PDO,PENDiff,SF%,SDiff,Sh%,Sv,goal_diff_target
0,2,2008020003,20081005,20082009,NYR,TBL,-125.0,105.0,2.0,1.0,...,0.547778,2.00,33.34,0.24,6.0,32.26,40.00,0.116144,0.12,1.0
1,3,2008020004,20081005,20082009,PIT,OTT,-110.0,110.0,1.0,3.0,...,2.919347,1.86,14.28,9.52,2.0,-7.70,-9.30,4.761905,4.76,-2.0
2,15,2008020016,20081011,20082009,TOR,MTL,110.0,-130.0,1.0,6.0,...,-1.953973,1.00,10.00,4.15,0.0,0.19,-2.00,6.773399,-2.63,-5.0
3,16,2008020017,20081011,20082009,OTT,DET,115.0,-135.0,2.0,3.0,...,2.840909,1.48,14.55,6.42,-5.0,-5.65,-7.52,4.272014,2.14,-1.0
4,17,2008020018,20081011,20082009,NYI,STL,-125.0,105.0,5.0,2.0,...,0.366300,-4.00,-38.10,-16.69,-1.0,-2.73,-3.00,-18.881119,2.19,3.0
5,19,2008020020,20081011,20082009,WSH,CHI,-140.0,120.0,4.0,2.0,...,-11.618590,-1.00,3.03,-5.74,-2.0,5.65,9.00,3.052326,-8.79,2.0
6,21,2008020022,20081011,20082009,PIT,NJD,-135.0,115.0,1.0,2.0,...,-3.787879,-1.48,-21.22,-4.53,5.0,-3.14,-3.48,1.300170,-5.83,-1.0
7,22,2008020023,20081011,20082009,TBL,CAR,-110.0,110.0,3.0,4.0,...,4.353741,-3.00,-26.67,-12.88,0.0,-8.78,-11.00,-20.000000,7.12,-1.0
8,23,2008020024,20081011,20082009,NSH,DAL,-125.0,105.0,3.0,1.0,...,0.816993,-2.07,-15.87,-8.02,-8.0,-3.45,-3.71,-3.812317,-4.21,2.0
9,25,2008020026,20081011,20082009,CGY,VAN,-145.0,125.0,4.0,5.0,...,-16.666667,-12.00,-100.00,-52.18,-6.0,4.16,4.00,-26.086957,-26.09,-1.0


In [7]:
##helper fns 

def fav_pred(x):
    if x < 0:
        return 1
    if x>0:
        return 0
    if x==0:
        print("O odds detected, nan returned")
        return np.NaN
v_fav_pred = np.vectorize(fav_pred)


def implied_proba(odds):
    if odds > 0: 
        return 100/(odds+100)    #bet 100 to get 100+odds; profit = odds
    
#def fav_implied_proba(odds):
    if odds < 0:
        return (-odds)/(-odds + 100)   #bet |odds| to get 100+|odds|; profit = 100
    
v_impl_proba = np.vectorize(implied_proba)

def are_equal(x1,x2):  # 1 if nums =1, 0, o/w  #delta 
    return int(x1 == x2)
v_are_equal = np.vectorize(are_equal)

In [8]:
print(X.shape)

filt_no_early  = (X['mp_date'].apply(lambda x : x% 10**4) < 900) | (1100 < X['mp_date'].apply(lambda x : x% 10**4))
X= X.loc[filt_no_early, : ].copy()  ## keep games < 900 and > 1100
print(X.shape)
filt_no_fav = (v_fav_pred(X['home_odds']) != v_fav_pred(X['away_odds'])).copy() 
X.loc[filt_no_fav, :]['season'].value_counts()
X = X.loc[ filt_no_fav , : ].copy()  #1000 plus rows across all seasons 
print(X.shape)


(10385, 27)
(9104, 27)
(7968, 27)


In [9]:
#woof two 10% loses
X.reset_index(inplace = True)    


In [10]:
##split data 

#X = remove_early(X).copy()
#did not remove early seasons yet


x = np.array(X.loc[(X['season'] <= 20152016), x_cols].copy())
  #features test-train
Y = X.loc[(X['season'] <= 20152016), y_cols].copy()
   #targets
y = np.array(Y['home_win']).reshape(-1,1)


              
x_16 = X.loc[(X['season'] == 20162017), x_cols].copy()  #features test-train
Y_16 = X.loc[(X['season'] == 20162017), y_cols].copy()   #targets
y_16 = np.array(Y_16['home_win']).reshape(-1,1)

x_17 = X.loc[(X['season'] > 20162017), x_cols].copy()  #features test-train
Y_17 = X.loc[(X['season'] > 20162017), y_cols].copy()   #targets
y_17 = np.array(Y_17['home_win']).reshape(-1,1)




In [11]:
X.loc[(X['season'] >= 20162017), :]['season'].value_counts()

20162017    928
20172018    907
Name: season, dtype: int64

In [12]:
X.loc[(X['season'] <= 20152016), :]['season'].value_counts()

20082009    940
20142015    940
20152016    905
20132014    855
20112012    853
20102011    821
20092010    819
Name: season, dtype: int64

In [13]:
X.loc[(X['season'] <= 20152016), :]['season'].value_counts().sum()

6133

In [14]:
print(X.shape)
print(x.shape, Y.shape, y.shape)
print(x_16.shape, y_16.shape)
print(x_17.shape, y_17.shape)
print('sum ', y.shape[0], y_16.shape[0], y_17.shape[0], ' is: ', y.shape[0] + y_16.shape[0]+y_17.shape[0])

(7968, 28)
(6133, 14) (6133, 13) (6133, 1)
(928, 14) (928, 1)
(907, 14) (907, 1)
sum  6133 928 907  is:  7968


In [15]:
Y['season'].value_counts()

20082009    940
20142015    940
20152016    905
20132014    855
20112012    853
20102011    821
20092010    819
Name: season, dtype: int64

In [16]:
Y_16['season'].value_counts()

20162017    928
Name: season, dtype: int64

In [17]:
Y_17['season'].value_counts()

20172018    907
Name: season, dtype: int64

In [62]:
x

array([[ -1.07      ,   3.88836773,  -1.50203584, ...,   0.        ,
          6.54126895,  -2.47      ],
       [ -5.33      ,   0.32215846,   0.79686845, ...,  -8.69      ,
          0.77548535,   2.28      ],
       [-11.39      ,   1.29385506,  -0.7319349 , ..., -10.51      ,
          2.1650334 ,  -2.47      ],
       ...,
       [  0.18      ,   0.34434966,   0.08231936, ...,   0.05      ,
          1.10672297,  -0.19      ],
       [  4.45      ,  -0.30295542,   0.7356406 , ...,   4.42      ,
         -0.24102006,   1.24      ],
       [  3.87      ,   0.24824888,   0.401685  , ...,   6.        ,
          0.33648903,   0.39      ]])

In [18]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler 
std_scal = StandardScaler()
mm_scal = MinMaxScaler()



##no tuning 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)



##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0, max_iter = 1000)
svc = SVC()

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=3, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()

##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    model.fit(x_train_sc, y_train.ravel())
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    y_predt= model.predict(x_train_sc)  
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    acct = accuracy_score(y_train, y_predt)
    f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], 'no tuning TEST: ', acc, f1, 'training : ', acct, f1t)
  
    


SVC() no tuning TEST:  0.5843520782396088 0.6867321867321868 training :  0.5966163880962088 0.7007409647663692
RidgeClassif no tuning TEST:  0.5704971475142624 0.6662444585180494 training :  0.5858132898491643 0.684863523573201
RandomForest no tuning TEST:  0.5729421352893236 0.6789215686274509 training :  0.5970240521810029 0.7015849056603773
GaussianNB() no tuning TEST:  0.5713121434392828 0.6210374639769453 training :  0.5699143905421933 0.6295646067415731
LogisticRegr no tuning TEST:  0.5688671556642216 0.6658243840808592 training :  0.5862209539339585 0.6844886540254895
BaggingClass no tuning TEST:  0.5395273023634882 0.563031709203403 training :  0.9826742763962495 0.984360625574977
GradientBoos no tuning TEST:  0.5770171149144254 0.6653771760154739 training :  0.6553200163065634 0.7295698064928835
[10:28:06] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the 

/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifie no tuning TEST:  0.5525672371638142 0.6168876482903002 training :  0.9641255605381166 0.9683339330694493


In [19]:
#check on 2016
x_16_sc = std_scal.transform(x_16).copy()
x_test2 = std_scal.transform(x_16).copy()
y_test2 = y_16.copy()


##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], '20162017, no tuning, TEST: ', acc, f1) #, 'training : ', acct, f1t)

SVC() 20162017, no tuning, TEST:  0.5980603448275862 0.698464025869038
RidgeClassif 20162017, no tuning, TEST:  0.603448275862069 0.6983606557377049
RandomForest 20162017, no tuning, TEST:  0.6012931034482759 0.7049441786283892
GaussianNB() 20162017, no tuning, TEST:  0.5862068965517241 0.6377358490566037
LogisticRegr 20162017, no tuning, TEST:  0.6012931034482759 0.696223316912972
BaggingClass 20162017, no tuning, TEST:  0.5269396551724138 0.5561172901921132
GradientBoos 20162017, no tuning, TEST:  0.5829741379310345 0.6766917293233083
XGBClassifie 20162017, no tuning, TEST:  0.5484913793103449 0.6221821460775473


In [20]:
#check on 2017

x_test2 = std_scal.transform(x_17).copy()
y_test2 = y_17.copy()


##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], '20172018, no tuning, TEST: ', acc, f1) #, 'training : ', acct, f1t)

SVC() 20172018, no tuning, TEST:  0.619625137816979 0.7331786542923435
RidgeClassif 20172018, no tuning, TEST:  0.5898566703417861 0.6985413290113452
RandomForest 20172018, no tuning, TEST:  0.6163175303197354 0.7318952234206471
GaussianNB() 20172018, no tuning, TEST:  0.5843439911797134 0.6544454628780935
LogisticRegr 20172018, no tuning, TEST:  0.5931642778390298 0.7016976556184318
BaggingClass 20172018, no tuning, TEST:  0.5071664829106945 0.5443425076452598
GradientBoos 20172018, no tuning, TEST:  0.6229327453142227 0.7237479806138934
XGBClassifie 20172018, no tuning, TEST:  0.5633958103638368 0.6476868327402135


In [ ]:
##complicated to get the indeces from the train test split ... so let's check on x_16 




In [61]:
x_test

array([[-4.33      , -0.08264702, -0.67063391, ..., -6.79      ,
         0.01095635, -0.87      ],
       [-5.01      , -1.07226995, -0.39499567, ..., -4.1       ,
        -2.79217032, -0.57      ],
       [ 0.19      , -0.39910557,  0.35727194, ...,  2.36      ,
        -0.27785496, -0.82      ],
       ...,
       [-3.87      , -0.48053769, -0.35714556, ..., -4.73      ,
        -1.10148156, -0.61      ],
       [-3.59      ,  0.79118313,  0.17763555, ..., -2.53      ,
         1.00100317,  0.38      ],
       [ 4.58      ,  0.15531033,  0.72882698, ...,  5.52      ,
         0.34114804,  1.25      ]])

In [60]:
pd.DataFrame(x_test_sc)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-1.048645,-0.122505,-0.844369,-1.270644,0.002901,-0.764105,-1.264077,-1.176017,-0.432751,-0.247588,-1.414627,-1.433510,-0.017484,-0.604359
1,-1.207517,-1.317469,-0.518039,-0.893779,-1.777327,-0.483778,-1.977181,-1.691767,-1.552153,-0.082129,-0.851693,-0.878860,-1.942587,-0.414321
2,0.007389,-0.504627,0.372575,0.094029,-0.288581,-0.044112,0.014129,-0.145910,-0.544246,-0.214496,0.446838,0.453126,-0.215831,-0.572686
3,2.287673,-0.373670,0.074809,1.992399,-0.262713,0.235395,1.103968,1.189465,0.191616,1.837191,1.686238,1.743873,-0.189859,0.440852
4,1.117158,-0.497231,0.582557,1.140357,-0.454269,0.498833,0.740689,0.689048,-0.125027,1.936466,1.376388,1.395413,-0.541096,0.320494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222,-0.936500,-0.815487,-0.036975,-0.699495,-0.711181,-0.482748,-1.129529,-1.078442,-1.119556,0.215696,-0.295856,-0.276786,-0.817085,-0.832405
1223,0.521387,-1.203343,0.671727,0.517710,-1.205224,0.623805,-0.080054,-0.091547,-0.388154,0.116421,0.541449,0.543849,-1.165795,0.529537
1224,-0.941172,-0.602956,-0.473228,-1.191058,-0.561975,-0.325627,-1.277531,-1.241531,-0.816291,-1.141064,-0.882442,-1.008759,-0.781473,-0.439659
1225,-0.875754,0.932640,0.159903,-0.643316,0.791510,0.060393,0.121768,0.121723,0.561777,-0.578505,-0.589149,-0.555142,0.662451,0.187468


In [ ]:
#Now do same for model on x_test
df_base2 = x_test_sc






###Part 2 Evaluate betting ,... "tune" on x_16 ... test again on x_17




In [52]:
df_base.head(2)

,Unnamed: 0,game_id,mp_date,season,home_team,away_team,home_odds,away_odds,home_goals,away_goals,goal_diff_target,home_win,settled_in,fav_pred
0,161,2008020162,20081102,20082009,PHI,EDM,-170.0,150.0,4.0,5.0,-1.0,0.0,REG,1
1,162,2008020163,20081102,20082009,CAR,TOR,-170.0,150.0,6.0,4.0,2.0,1.0,REG,1


In [56]:
###baseline pretraining accuracy when model says to bet fav and bet und
###find some baselines from Y"
df_base = Y.copy()
df_base['fav_pred'] = v_fav_pred(df_base['home_odds'] ).copy()
df_base.shape

(6133, 14)

In [57]:
((df_base['home_win'] == 1)&(df_base['fav_pred'] == 1)).sum()

2819

In [58]:
fav_home_acc = ((df_base['home_win'] == 1)&(df_base['fav_pred'] == 1)).sum()/(df_base['fav_pred'] == 1).sum()
und_home_acc = ((df_base['home_win'] == 0)&(df_base['fav_pred'] == 0)).sum()/(df_base['fav_pred'] == 0).sum()
print(fav_home_acc, und_home_acc)


0.5949767834529337 0.567741935483871


In [120]:
def make_bet_df(model=rfc, x = x_16, y = y_16, Y = Y_16):
    dic = {}  #x_16, lgr 0.55 ... Request rfc 0.58 x_16 and 0.6 on x_17 (later)
    
    dic['model_name'] = str(model)[0:10]
    dic['actual_res'] = [ t[0] for t in list(y)]
    dic['model_pred'] = list(model.predict(x))
    
    
    dic['model_conf_1'] = [round(t,4) for t in model.predict_proba(x)[0:, 1]]
    dic['model_conf_0'] =[round(t,4) for t in model.predict_proba(x)[0:, 0]]
    
    dic['home_odds'] = list(Y['home_odds'])
    dic['away_odds'] = list(Y['away_odds'])
    
    df= pd.DataFrame(dic)
    
    
    #wrong! ... actually ok
    df['home_impl_proba'] = v_impl_proba(df['home_odds'])
    df['away_impl_proba'] = v_impl_proba(df['away_odds'])
    
    df['conf_1_sub_home_impl'] = df['model_conf_1'] - df['home_impl_proba']
    df['conf_0_sub_away_impl'] = df['model_conf_0'] - df['away_impl_proba']
    
    df['pre_acc_sub_home_impl'] = (acc_pre - df['home_impl_proba']).copy()
    df['pre_acc_sub_away_impl'] = (acc_pre  - df['away_impl_proba']).copy()
    
    df['fav_pred'] = v_fav_pred(df['home_odds'] )
    df['model_conf'] = (0.5)*(df['model_conf_1'] - df['model_conf_0'])
    df['model_conf'] = np.abs(df['model_conf'])
    return df   #df_bet_info

In [164]:
df_test17 = make_bet_df(model=rfc, x = x_17, y = y_17, Y = Y_17)
df_test= make_bet_df(model=rfc, x = x_16, y = y_16, Y = Y_16)   ##this is for model = rfc and season = 20162017; rfc dif well on both seasons

In [122]:
bet_fav = ((df_test['fav_pred'] ==  df_test['model_pred'])&(df_test['actual_res'] == df_test['model_pred'])).sum()/(df_test['fav_pred'] ==  df_test['model_pred']).sum()
bet_und =((df_test['fav_pred'] !=  df_test['model_pred'])&(df_test['actual_res'] == df_test['model_pred'])).sum()/(df_test['fav_pred'] !=  df_test['model_pred']).sum()
print(bet_fav)
print(bet_und)

0.603494623655914
0.4891304347826087


In [126]:
bet_away_fav = ((df_test['fav_pred'] == 0)&(df_test['model_pred'] == 0)&(df_test['actual_res'] == 0)).sum()/((df_test['fav_pred'] == 0)&(df_test['model_pred'] == 0)).sum()
bet_away_und = ((df_test['fav_pred'] == 1)&(df_test['model_pred'] == 0)&(df_test['actual_res'] == 0)).sum()/((df_test['fav_pred'] == 1)&(df_test['model_pred'] == 0)).sum()
bet_home_fav = ((df_test['fav_pred'] == 1)&(df_test['model_pred'] == 1)&(df_test['actual_res'] == 1)).sum()/((df_test['fav_pred'] == 1)&(df_test['model_pred'] == 1)).sum()
bet_home_und = ((df_test['fav_pred'] == 0)&(df_test['model_pred'] == 1)&(df_test['actual_res'] == 1)).sum()/((df_test['fav_pred'] ==0)&(df_test['model_pred'] == 1)).sum()

print(bet_away_fav)
print(bet_away_und)
print(bet_home_fav)
print(bet_home_und)

##can use these in x_17 betting model but not in the x_16 one ... except I'kk just say maybe don'tbet on away und

0.6114285714285714
0.46853146853146854
0.6010544815465729
0.5609756097560976


In [132]:

cols = ['model_conf','home_impl_proba', 'model_conf_1', 'conf_1_sub_home_impl', 'pre_acc_sub_home_impl' 
                      ,'away_impl_proba', 'model_conf_0', 'conf_0_sub_away_impl',   'pre_acc_sub_away_impl' ,  'model_pred', 'fav_pred', 'actual_res', 'home_odds', 'away_odds',]

In [135]:
df_test.columns

Index(['model_name', 'actual_res', 'model_pred', 'model_conf_1',
       'model_conf_0', 'home_odds', 'away_odds', 'home_impl_proba',
       'away_impl_proba', 'conf_1_sub_home_impl', 'conf_0_sub_away_impl',
       'pre_acc_sub_home_impl', 'pre_acc_sub_away_impl', 'fav_pred',
       'model_conf'],
      dtype='object')

In [129]:
(df_test['conf_0_sub_away_impl'] >=-0.1)

0       True
1       True
2      False
3       True
4       True
       ...  
923     True
924     True
925    False
926     True
927     True
Name: conf_0_sub_away_impl, Length: 928, dtype: bool

In [137]:
df_test.loc[(df_test['model_conf'] >=0.1), cols]

,model_conf,home_impl_proba,model_conf_1,conf_1_sub_home_impl,pre_acc_sub_home_impl,away_impl_proba,model_conf_0,conf_0_sub_away_impl,pre_acc_sub_away_impl,model_pred,fav_pred,actual_res,home_odds,away_odds
0,0.1411,0.682540,0.6411,-0.041440,-0.109598,0.350877,0.3589,0.008023,0.222065,1.0,1,1.0,-215.0,185.0
2,0.1538,0.523810,0.6538,0.129990,0.049133,0.476190,0.3462,-0.129990,0.096752,1.0,1,0.0,-110.0,110.0
5,0.2232,0.629630,0.7232,0.093570,-0.056687,0.400000,0.2768,-0.123200,0.172942,1.0,1,1.0,-170.0,150.0
6,0.2416,0.696970,0.7416,0.044630,-0.124028,0.344828,0.2584,-0.086428,0.228115,1.0,1,1.0,-230.0,190.0
7,0.1391,0.642857,0.6391,-0.003757,-0.069915,0.384615,0.3609,-0.023715,0.188327,1.0,1,1.0,-180.0,160.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,0.1475,0.619772,0.6475,0.027728,-0.046830,0.411523,0.3525,-0.059023,0.161420,1.0,1,1.0,-163.0,143.0
919,0.2311,0.622642,0.7311,0.108458,-0.049699,0.408163,0.2689,-0.139263,0.164779,1.0,1,1.0,-165.0,145.0
922,0.2243,0.672131,0.7243,0.052169,-0.099189,0.363636,0.2757,-0.087936,0.209306,1.0,1,1.0,-205.0,175.0
925,0.1269,0.565217,0.6269,0.061683,0.007725,0.476190,0.3731,-0.103090,0.096752,1.0,1,0.0,-130.0,110.0


In [131]:
df_test.loc[10:20,   [ 'model_conf', 'model_pred', 'fav_pred', 'actual_res', 'home_odds', 'away_odds','home_impl_proba', 'model_conf_1', 'conf_1_sub_home_impl', 'pre_acc_sub_home_impl' 
                      ,'away_impl_proba', 'model_conf_0', 'conf_0_sub_away_impl',   'pre_acc_sub_away_impl' ],]

,model_conf,model_pred,fav_pred,actual_res,home_odds,away_odds,home_impl_proba,model_conf_1,conf_1_sub_home_impl,pre_acc_sub_home_impl,away_impl_proba,model_conf_0,conf_0_sub_away_impl,pre_acc_sub_away_impl
10,0.1154,1.0,1,1.0,-170.0,150.0,0.629630,0.6154,-0.014230,-0.056687,0.400000,0.3846,-0.015400,0.172942
11,0.0664,0.0,0,1.0,125.0,-145.0,0.444444,0.4336,-0.010844,0.128498,0.591837,0.5664,-0.025437,-0.018895
12,0.2190,1.0,1,0.0,-215.0,185.0,0.682540,0.7190,0.036460,-0.109598,0.350877,0.2810,-0.069877,0.222065
13,0.0905,0.0,0,1.0,105.0,-125.0,0.487805,0.4095,-0.078305,0.085137,0.555556,0.5905,0.034944,0.017387
14,0.0197,0.0,1,0.0,-170.0,150.0,0.629630,0.4803,-0.149330,-0.056687,0.400000,0.5197,0.119700,0.172942
15,0.2220,1.0,1,1.0,-215.0,185.0,0.682540,0.7220,0.039460,-0.109598,0.350877,0.2780,-0.072877,0.222065
16,0.0841,0.0,1,0.0,-125.0,105.0,0.555556,0.4159,-0.139656,0.017387,0.487805,0.5841,0.096295,0.085137
17,0.1231,1.0,1,1.0,-170.0,150.0,0.629630,0.6231,-0.006530,-0.056687,0.400000,0.3769,-0.023100,0.172942
18,0.0157,0.0,1,1.0,-185.0,165.0,0.649123,0.4843,-0.164823,-0.076181,0.377358,0.5157,0.138342,0.195584
19,0.2175,1.0,1,0.0,-150.0,130.0,0.600000,0.7175,0.117500,-0.027058,0.434783,0.2825,-0.152283,0.138160


In [21]:
model = rfc

y_pred= model.predict(x_test_sc)  #this is <= 20152016 data
acc_pre = accuracy_score(y_test, y_pred)  #this x_test_sc needs to be defined already above during training session on <=2015
acc_pre

0.5729421352893236

In [138]:
df = Y_16.loc[:, ['home_team', 'away_team','home_odds', 'away_odds', 'goal_diff_target', 'home_win',]].head(20).copy()

In [193]:
find_pay_off(df_test17)

518.0 1.0255487053020962
3844.0 1.600625
3963.0 1.230406976744186
1720.0 1.0182722134874431
138007.0 10045.0 1.0727861630207163


In [192]:
find_pay_off(df_test)

592.0 1.022470204205572
3079.0 1.7509756097560976
7392.0 1.5169230769230768
-3187.0 0.967852891928423
143884.0 7876.0 1.0547385393789441


In [172]:
#(1/df_test['home_impl_proba'])*100
df2 = df_test.copy()
(df2['actual_res']*(  100*(1/df2['home_impl_proba']) + 100    )-100)

0      146.511628
1      180.000000
2     -100.000000
3     -100.000000
4     -100.000000
          ...    
923    157.142857
924   -100.000000
925   -100.000000
926    155.555556
927    130.769231
Length: 928, dtype: float64

In [191]:



def find_pay_off(df_test):

    df = df_test.copy()
    
    ##bet fav_home 
    df2 = df.loc[ (df_test['fav_pred'] !=  df_test['model_pred'])&(df_test['model_pred'] ==1), :]
    df2['bet'] = 100
    df2['win'] = df2['actual_res']*(  100*(1/df2['home_impl_proba']) + 100    )
    df2['profit'] = df2['win'] - df2['bet']
    
    ##bet fav_away
    df3= df.loc[ (df_test['fav_pred'] !=  df_test['model_pred'])&(df_test['model_pred'] == 0), :]
    df3['bet'] = 100
    df3['win'] = (1-df3['actual_res'])*(100*(1/df3['away_impl_proba']) + 100 )
    df3['profit'] = df3['win'] - df3['bet']
    
    ##bet und_home 
    df4 = df.loc[ (df_test['fav_pred'] ==  df_test['model_pred'])&(df_test['model_pred'] ==1), :]
    df4['bet'] = -df4['home_odds']
    df4['win'] = df4['actual_res']*(-df['home_odds'] + 100) 
    df4['profit'] = df4['win'] - df4['bet']
    
    df5 =df.loc[ (df_test['fav_pred'] ==  df_test['model_pred'])&(df_test['model_pred'] == 0), :]
    df5['bet'] =  -df5['away_odds']
    df5['win'] = (1-df5['actual_res'])*(-df5['away_odds'] + 100) 
    df5['profit'] = df5['win'] - df5['bet']
    
    print(df5['profit'].sum(), (df5['profit'].sum() +   df5['bet'].sum())/df5['bet'].sum()        )
    print(df2['profit'].sum(), (df2['profit'].sum() +   df2['bet'].sum())/df2['bet'].sum()        )
    print(df3['profit'].sum(), (df3['profit'].sum() +   df3['bet'].sum())/df3['bet'].sum()        )                                    
    print(df4['profit'].sum(), (df4['profit'].sum() +   df4['bet'].sum())/df4['bet'].sum()        )
    profit_t = df2['profit'].sum()+df3['profit'].sum()+df4['profit'].sum() +df5['profit'].sum()
    bet_t = df2['bet'].sum()+df3['bet'].sum() +df4['bet'].sum()+df5['bet'].sum()
    
    print(bet_t, profit_t, (profit_t+bet_t)/bet_t    )                                 
                                    
  
    

In [29]:
df_test.columns


Index(['model_name', 'actual_res', 'model_pred', 'model_conf_1',
       'model_conf_0', 'home_odds', 'away_odds', 'home_impl_proba',
       'away_impl_proba', 'conf_1_sub_home_impl', 'conf_0_sub_away_impl',
       'pre_acc_sub_home_impl', 'pre_acc_sub_away_impl', 'fav_pred'],
      dtype='object')

In [ ]:
df_bet_info.describe()

In [ ]:
#df_bet_info

min_proba =0
pre_acc_conf = -0.21
model_conf = 0
row = 0
model_not_conf = (0.5 - model_conf<= df_bet_info.loc[row,'model_conf_1'])&(df_bet_info.loc[row,'model_conf_1'] <= 0.5 + model_conf)
model_conf = (not model_not_conf)
model_conf
odds_fav = min(df_bet_info.loc[row, 'away_odds'], df_bet_info.loc[row, 'home_odds'])
odds_und = max(df_bet_info.loc[row, 'away_odds'], df_bet_info.loc[row, 'home_odds'])
odds_und
fav_home = bool(df_bet_info.loc[row, 'fav_pred'])    
df_bet_info.loc[row, 'fav_pred']
bet_fav = df_bet_info.loc[row, 'model_pred'] == df_bet_info.loc[row, 'fav_pred']  #True of False 
bet_und = df_bet_info.loc[row, 'model_pred'] != df_bet_info.loc[row, 'fav_pred'] 
bet_und
bet_fav and fav_home and model_conf and (df_bet_info.loc[row, 'conf_1_sub_home_impl'] >= min_proba) and (df_bet_info.loc[row, 'pre_acc_sub_home_impl'] >= pre_acc_conf)
bet_fav           
fav_home
model_conf
(df_bet_info.loc[row, 'conf_1_sub_home_impl'] >= min_proba)
counter = 0
#for row in df_bet_info.index:
(df_bet_info.loc[:, 'pre_acc_sub_home_impl'] >= pre_acc_conf).sum()

In [ ]:
df_bet_info.loc[row, 'pre_acc_sub_home_impl']
df_bet_info

In [ ]:
df_bet_info.loc[row, 'model_pred'] ==  df_bet_info.loc[row, 'actual_res']

In [ ]:
def find_profit(df_bet_info, min_proba =-10, pre_acc_conf = -10, above_05 = 0 ):
    df = df_bet_info.copy()
   
    for row in df_bet_info.index:
        #model_not_conf = (0.5 - model_conf< df_bet_info.loc[row,'model_conf_1'])&(df_bet_info.loc[row,'model_conf_1'] < 0.5 + model_conf)
        model_conf = not((0.5 - above_05 <= df_bet_info.loc[row,'model_conf_1']) & (df_bet_info.loc[row,'model_conf_1']<=0.5 + above_05))
        odds_fav = min(df_bet_info.loc[row, 'away_odds'], df_bet_info.loc[row, 'home_odds'])
        odds_und = max(df_bet_info.loc[row, 'away_odds'], df_bet_info.loc[row, 'home_odds'])
        fav_home = bool(df_bet_info.loc[row, 'fav_pred'])    
        
        bet_fav = df_bet_info.loc[row, 'model_pred'] == df_bet_info.loc[row, 'fav_pred']  #True of False 
        bet_und = df_bet_info.loc[row, 'model_pred'] != df_bet_info.loc[row, 'fav_pred'] 
        #bet fav:
        if bet_fav and fav_home and model_conf and (df_bet_info.loc[row, 'conf_1_sub_home_impl'] >= min_proba) and (df_bet_info.loc[row, 'pre_acc_sub_home_impl'] >= pre_acc_conf):
            bet = -odds_fav
            pay_off = 100
            if df_bet_info.loc[row, 'model_pred'] ==  df_bet_info.loc[row, 'actual_res']:
                df.loc[row,'profit'] = pay_off
                df.loc[row,'bet'] = bet
            else:
                df.loc[row,'profit'] = -bet
            
        elif bet_fav and (not fav_home) and model_conf and (df_bet_info.loc[row, 'conf_0_sub_away_impl'] >= min_proba) and  (df_bet_info.loc[row, 'pre_acc_sub_away_impl'] >=pre_acc_conf):
            bet = -odds_fav
            pay_off = 100
            if df_bet_info.loc[row, 'model_pred'] ==  df_bet_info.loc[row, 'actual_res']:
                df.loc[row,'profit'] = pay_off
                df.loc[row,'bet'] = bet
            else:
                df.loc[row,'profit'] = -bet
        elif bet_und and fav_home and model_conf and (df_bet_info.loc[row, 'conf_1_sub_home_impl'] >= min_proba) and (df_bet_info.loc[row, 'pre_acc_sub_home_impl'] >= pre_acc_conf):
            bet = 100
            pay_off = odds_und
            if df_bet_info.loc[row, 'model_pred'] ==  df_bet_info.loc[row, 'actual_res']:
                df.loc[row,'profit'] = pay_off
                df.loc[row,'bet'] = bet
            else:
                df.loc[row,'profit'] = -bet
        elif bet_und and (not fav_home) and model_conf and(df_bet_info.loc[row, 'conf_0_sub_away_impl'] >= min_proba) and  (df_bet_info.loc[row, 'pre_acc_sub_away_impl'] >=pre_acc_conf):
            bet = 100
            pay_off = odds_und
            if df_bet_info.loc[row, 'model_pred'] ==  df_bet_info.loc[row, 'actual_res']:
                df.loc[row,'profit'] = pay_off
                df.loc[row,'bet'] = bet
            else:
                df.loc[row,'profit'] = -bet
        else:
            df.loc[row,'profit'] = 0
            df.loc[row,'bet'] = 0
        
    
    return df
            
            

In [ ]:
find_profit(df_bet_info, min_proba =-10, pre_acc_conf = -10, above_05 = 0)['profit'].sum()

In [ ]:
df = find_profit(df_bet_info, min_proba =-1000, pre_acc_conf = -100, model_conf =0).copy()
print(df['profit'].sum())
print((df['profit']!=0).sum())
print((df['profit'].sum()+df['bet'].sum())/df['bet'].sum())

In [ ]:
conf = 0.02
(~((0.5 - conf <= df['model_conf_1']) & (df['model_conf_1']<=0.5 + conf))).sum()

In [ ]:


print(find_profit(df_bet_info, min_proba =0, pre_acc_conf = -0.2, model_conf = -0.05)['profit'].sum())  #.value_counts())
print((find_profit(df_bet_info, min_proba =0, pre_acc_conf = -0.15, model_conf = 0 )['profit']==0).sum())


In [ ]:
df_bet_info['profit'] = 
            dic['profit'] = payoff
            dic['won'] = 1
        if df_bet_info.loc[row, 'model_pred'] !=  df_bet_info.loc[row, 'actual_res']:
            dic['bet'].append(bet)
            dic['payoff'] = payoff
            dic['won'] = 0

In [ ]:
df = df_bet_info.loc[df_bet_info['model_pred'] == df_bet_info['fav_pred'], :]
fav_odds = v_min(df['home_odds'],df['away_odds'])
df.loc[:,'fav_odds'] = fav_odds

In [ ]:
def min(a,b):
    if a<= b:
        return a
    if b < a:
        return b
v_min = np.vectorize(min)
def simple_bet_fav(df_bet_info):

    df = df_bet_info.loc[df_bet_info['model_pred'] == df_bet_info['fav_pred'], :].copy()
    filt = df['']
    fav_odds = v_min(df['home_odds'],df['away_odds'])
    df.loc[:, 'fav_odds'] = fav_odds
    df['bet_fav'] = -df['fav_odds']
    df['won_bet'] = v_are_equal(df['model_pred'], df['actual_res']  )
    df['profit'] = df['won_bet']*100 - (1-df['won_bet'])*df['bet_fav']
    df['accuracy'] = accuracy_score(df['actual_res'], df['model_pred']    ) 
    
    df['total_profit'] = df['profit'].sum()
    
    return df

In [ ]:

simple_bet_fav(df_bet_info)

In [ ]:
simple_bet_fav(df_bet_info).iloc[:, -14:].describe()

In [ ]:
df = simple_bet_fav(df_bet_info)
df = df.loc[:, ['model_name', 'actual_res', 'model_pred', 'model_conf_1',
       'model_conf_0', 'home_odds', 'away_odds', 'home_impl_proba',
       'away_impl_proba','fav_pred',
       'fav_odds', 'bet_fav', 'won_bet', 'profit', 'accuracy']].copy()
df

In [ ]:
#simpler approach? 


y_pred= model.predict(x_test_sc)  #this is <= 20152016 data
acc_pre = accuracy_score(y_test, y_pred)  #this x_test_sc needs to be defined already above during training session on <=2015

##bet the spread mofo!

#conf_min >=0 is how far from 0.5 you want conf_1 or conf_0 to be (if one is close to 0.5 so is the other)

## pre_acc_above_break_even  = 0.1 means you would ask pre_acc >= break_even_prob + 0.1   pre_acc is accuracy of model on pretraining

##model_conf_above_break_even  = 0.1 means you would ask predict_proba >= break_even_prob + 0.1   pred_proba is the current model's 
#estimate on how likely it thimks the anwer is 1 (or 0 if it is predicting 0

def make_pay_off_df(df_bet_info=df_bet_info, model_conf_min = 0 , min_pre_acc_above_break_even = 0, min_model_conf_above_break_even = 0, bet_fav = True, bet_und = True): 
    ##return df with columns for each season in question:
    #type_bet,  total_invested, total_earned, profit, ROI, 
    ##rows should be: avg for season in question let's do x_16 for now
    ###filters for df ..
    
    #df= df_bet_info.loc[: , 'model_pred'].copy() ##set up stub df
    
    #set up basic 0 and 1 cases ... 
    HA_bet = df_bet_info['model_pred'].copy()  #0 or 1
    fav_pred = df_bet_info['fav_pred'].copy()  #0 or 1 ... need 4 terms one for each combo
    actual_res= df_bet_info['actual_res'].copy()  
    
    ##STEP 1
    ##decide on bet_fav bet_und or do both strategy
    
    if bet_fav and ~bet_und:
        filt_bet  = (HA_bet == fav_pred).copy()
    if ~bet_fav and bet_und:
        filt_bet = (HA_bet != fav_pred).copy()
    if ~bet_fav and ~bet_und:
        filt_bet = (HA_bet != HA_bet).copy()  #all False
    if  bet_fav and bet_und:
        filt_bet = (HA_bet == HA_bet).copy() ##all True no filter
    
    #filt_bet_fav = (HA_bet == fav_pred)|(~bet_fav)  # if bet_fav = False, want this to be all True; if bet_fav = True, want to use this filter as is; do OR not bet_fav
    #filt_bet_und = (HA_bet != fav_pred)|(~bet_und)  #same deal ... if bet_und = True, want this filter to take effect; if False, want it to be all True
    #filt_fav_und = filt_bet_fav & filt_bet_und   ##not possible to do both stragies ... 
    
    df = df_bet_info.loc[filt_bet, :].copy()  #set betting fav/und strategy
    

    
    ##STEP 2  ## impose the restrictions on confidence levels, if any; to keep it simple we impose same constraint whether H/A or fav/und
    
    ##! I think filtering by betting fav or und FIRST before calculating confidence levels is important
    ## therfore USE df and not df_bet_info (unfiltered) below
    
    
   
    
    #conf_min = 0; first we define model not confident to mean proba is close to 0.5 (both 1 and 0 conf will both be close to 0.5)
    #eg within conf_min =0.05  of 0.5 means not confident
    model_not_conf = (0.5 - model_conf_min <= df['model_conf_1'])&(df['model_conf_1'] <= 0.5 + model_conf_min).copy() 
    
    filt_model_is_conf = (~model_not_conf).copy()
    filt_= df['conf_1_sub_home_impl'] >=  min_pre_acc_above_break_even
    
    ##it is tricky because if you are betting on home team you need your conf_1 to be high; but it betting away team, you need conf_0 to be high
    conf_above_impl_proba_home_bet = (HA_bet*df['conf_1_sub_home_impl'] >= HA_bet*min_model_conf_above_break_even).copy()    #if HA_bet =0, reduced to 0>=0 (all True)
    conf_above_impl_proba_away_bet = ((1-HA_bet)*df['conf_0_sub_away_impl'] >=(1-HA_bet)*min_model_conf_above_break_even).copy() 
    
    filt_conf_above_impl_proba = conf_above_impl_proba_home_bet&conf_above_impl_proba_away_bet  #one of these will be all True
    
     
    pre_acc_above_impl_proba_home_bet = (HA_bet*df['pre_acc_sub_home_impl'] >= HA_bet*min_pre_acc_above_break_even).copy()    #if HA_bet =0, reduced to 0>=0 (all True)
    pre_acc_above_impl_proba_away_bet = ((1-HA_bet)*df['pre_acc_sub_home_impl'] >=(1-HA_bet)* min_pre_acc_above_break_even).copy() 
    
    filt_acc_above_impl_proba = pre_acc_above_impl_proba_home_bet&pre_acc_above_impl_proba_away_bet
    
    filt_conf = filt_model_is_conf&filt_conf_above_impl_proba&filt_acc_above_impl_proba
    
    #second restriction:
    df = df.loc[filt_conf, :].copy() 
    
    
    
    
    
    return df
    

In [ ]:

    
    
    #cases:    
    #bet = 0
        ##away_odds <0 (ie df['fav_pred'] = 0) ---> bet -away_odds profit  100
       ##away_odds >0 (ie df['fav_pred'] = 1) ---> bet 100        profit away_odds
    
    bet_A_A = (1- bet_HA)*(1-fav_pred)*(-df['away_odds']).copy()
    win_A_A = ((1- bet_HA)*(1-fav_pred)(1-actual_res)*100).copy()   
        
    bet_A_H = ((1-bet_HA)*fav_pred*100).copy()  
    win_A_H = ((1-bet_HA)*fav_pred*df['away_odds']*actual_res).copy()

    ##bet = 1  (ie df['model_pred'] = 1)
        ##home_odds >0 (ie df['fav_pred'] = 0) ---> bet 100        profit home_odds
        ##home_odds <0 (ie df['fav_pred'] = 1) ---> bet -home_odds profit  100
        
    bet_H_A = (bet_HA*(1-fav_pred)*100).copy()
    win_H_A = (bet_HA*(1-fav_pred)*df['home_odds']).copy()  
    
    bet_H_H = (bet_HA*fav_pred*(-df['home_odds']).copy()  
    win_H_H = (bet_HA*fav_pred*100).copy()  
    
    
    df['potential_profit'] = win_H_H + win_H_A + win_A_H + win_A_A 
    
    #if bet and result are same then  (1+ actual_result - bet_HA)  = 1 
    df['actual_profit'] = (bet_HA*actual_result)
               
               (-df['bet_amount'])(1+ actual_result + bet_HA)    +   (df['potential_profit'])(1+ actual_result - bet_HA)  
               
    
    df['profit'] = df['pay_out'] - df['bet_amount']       
    df['cumul_profit'] = 0
    
    for i in df.index:
        df.loc[i, 'cumul_profit'] = df.loc[:i, 'profit'].sum()
    
    df['total_profit'] = df['profit'].sum()    
    df['total_bet'] = df['bet'].sum()    
    df['total_ROI'] = df['profit'].sum() /df['bet'].sum()   
    
    #df.drop(columns = ['ones', 'non_pred'], inplace = True)
    return df
    

In [ ]:
df_pay = make_pay_off_df(df_bet_info=df_bet_info, model_conf_min = 0 , min_pre_acc_above_break_even = -10**5, min_model_conf_above_break_even = -10**6, bet_fav = True, bet_und = True) 
    ##return df wi
df_pay

#can do version where we get y regr version at beinning ... do all teh regression stuff ... 
then for later cells do y = v_win(y) for classifiers ... 

##regression models now ...

y = Y['goal_diff_target'].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)


lr = Ridge(alpha=50000) 
rfr = RandomForestRegressor(max_depth=4, random_state=0)
xgbr = XGBRegressor()


##quick checks 
for model in [lr, rfr, xgbr]:
    model.fit(x_train_sc, y_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    print(y_pred[0:5])
    y_predw = v_win(y_pred)
    y_predt= model.predict(x_train_sc)  
    y_predwt = v_win(y_predt) 
    y_trainw = v_win(y_train)
    y_testw = v_win(y_test)  #same as usual win/loss
    acc = accuracy_score(y_testw, y_predw)
    f1 = f1_score(y_testw,y_predw)
    acct = accuracy_score(y_trainw, y_predwt)
    f1t = f1_score(y_trainw,y_predwt)
    
  
    print(str(model)[0:20], 'TEST: ', acc, f1 ,'training : ', acct, f1t)
    

In [ ]:
#Let's go with lgr, lgr2 (tuned or not) for betting investigation

In [ ]:
x_16 = std_scal.transform(x_16).copy()


In [ ]:
svc2.predict_proba(x_16)

In [ ]:
gnb.predict_proba(x_16)[0:, 0]

In [ ]:
y_16

In [ ]:
##make betting strategy on x_16 test it on x_17  ... and other seasons if needed

In [ ]:
df_lgr.columns

In [ ]:
df_lgr['away_impl_proba'] = v_impl_proba(df_lgr['away_odds'])

df_lgr['home_impl_proba'] = v_impl_proba(df_lgr['home_odds'])


df_lgr['conf_1_sub_home_impl'] = df_lgr['lgr_conf_1'] - df_lgr['home_impl_proba']
df_lgr['conf_0_sub_away_impl'] = df_lgr['lgr_conf_0'] - df_lgr['away_impl_proba']

In [ ]:
df_lgr = df_lgr.loc[:, ['actual', 'lgr_pred', 'lgr_conf_1', 'home_impl_proba',        'conf_1_sub_home_impl', 'conf_0_sub_away_impl',    'lgr_conf_0', 'away_impl_proba', 
'home_odds',  'away_odds']].copy()

In [ ]:
df['away_odds']


In [ ]:
df_100.describe()

In [ ]:
df_lgr.loc[df_lgr.loc[df_lgr['home_odds'] == df_lgr['away_odds'], :]

In [ ]:
np.abs(-6)

In [ ]:
def v_fav(x):
    if x <0:
        return 1

In [ ]:
df['profit'].sum()

In [ ]:
cols = ['actual', 'lgr_pred', 'lgr_conf_1', 'home_impl_proba',
       'conf_1_sub_home_impl', 'conf_0_sub_away_impl', 
       'away_impl_proba',  'bet_HA',
       'bet', 'pay_out', 'profit', 'total_profit','cumul_profit', 'total_bet', 'total_ROI' ]

In [ ]:
df_100.shape

In [ ]:
df_100.loc[800:900,cols]

In [ ]:
df_100 = find_pay_off_bet_100(conf_1_at_least = 0 ,  conf_0_at_least =0 ,conf_thresh=0 , diff_home_C_1_at_least =0, diff_away_C_0_at_least =0, type_bet = "Bet_100"):  #or "bet_100"
    #

In [ ]:
df_100 = find_pay_off_bet_100(conf_1_at_least = 0, conf_0_at_least = 0, conf_thresh = -100,diff_home_C_1_at_least =0.2, diff_away_C_0_at_least =0.1, type_bet = "bet_100"  )
df_100['total_profit']

In [ ]:
df = find_pay_off(conf_1_at_least = 0.2,  conf_0_at_least = 0.2, diff_home_C_1_at_least = -10, diff_away_C_0_at_least = -10, type_bet = "get_100"  )
print(accuracy_score(df['actual'], df['lgr_pred'], ))
df['total_profit']                   #OR#                  #AND#                      ##OR##

In [ ]:
df_lgr.describe()

In [ ]:
accuracy_score(df_lgr['actual'], df_lgr['lgr_pred'], )

In [ ]:
print(df.loc[(df['bet_HA'] == 1)&(df['pay_out'] >0), ['profit']].sum(),
df.loc[(df['bet_HA'] == 1)&(df['pay_out'] >0), ['profit']].count(),
'avg loss ', df.loc[(df['bet_HA'] == 1)&(df['pay_out'] >0), ['profit']].sum()/df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].count()     )

print(df.loc[(df['bet_HA'] == 1)&(df['pay_out'] ==0), ['profit']].sum(),
df.loc[(df['bet_HA'] == 1)&(df['pay_out'] ==0), ['profit']].count(),
'avg loss ', df.loc[(df['bet_HA'] == 1)&(df['pay_out'] ==0), ['profit']].sum()/df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].count()     )

print(df.loc[(df['bet_HA'] == 0)&(df['pay_out'] >0), ['profit']].sum(),
df.loc[(df['bet_HA'] == 0)&(df['pay_out'] >0), ['profit']].count(),
'avg loss ', df.loc[(df['bet_HA'] == 0)&(df['pay_out'] >0), ['profit']].sum()/df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].count()     )

print(df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].sum(),
df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].count(),
'avg loss ', df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].sum()/df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']].count()     )


print(df['profit'].sum())
print('avg loss', df['profit'].sum()/df['profit'].count())


In [ ]:
(df.loc[(df['bet_HA'] == 0)&(df['pay_out'] ==0), ['profit']] <0).sum()

In [ ]:


dic2['lgr_pred'] = list(lgr.predict(x_16))
dic2['gnb_pred'] = list(gnb.predict(x_16))
dic2'svc_proba'] = list(svc.predict_proba(x_16)[:,1])


In [ ]:
predictions = gnb.predict(x_17)
actual = y_17
confusionMatrix = confusion_matrix(actual, predictions)
display(confusionMatrix)

tn = confusionMatrix[0][0]
fp = confusionMatrix[0][1]
fn = confusionMatrix[1][0]
tp = confusionMatrix[1][1]
actualYes = fn + tp 
actualNo = tn + fp
predictedYes = fp + tp
predictedNo = tn + fn

print('When home team wins, classifier predicts they will win %6.2f%% of the time' % (tp / actualYes * 100))
print('When home team loses, classifier predicts they will win %6.2f%% of the time' % (fp / actualNo * 100))
print('When home team loses, classifier predicts they will lose %6.2f%% of the time' % (tn / actualNo * 100))
print('When classifer predicts home team will win, home team actually wins %6.2f%% of the time' % (tp / predictedYes * 100))
print('When classifer predicts home team will lose, home team actually loses %6.2f%% of the time' % (tn / predictedNo * 100))

In [ ]:
## is the betting and calcs being done correctly?  62% accuracy should easily have a profit ... 


##try all models

##try regression 

##think about this "confidence diff " Leung mentioned 



##also ... sigh ... put in the FW% thing I guess ... [that was what I was struggling with calculating yesterday ]


In [ ]:
y_pred= model.predict(x_test_sc)  #this is <= 20152016 data
acc_pre_2016 = accuracy_score(y_test, y_pred)  #this x_test_sc needs to be defined already above during training session on <=2015

##bet the spread mofo!
#conf_min >=0 is how far from 0.5 you want conf_1 or conf_0 to be (if one is close to 0.5 so is the other) 
def make_pay_off_df(df_bet_info=df_bet_info, conf_min = 0 , acc_pre_2016_min_amount_above_home_impl_  =-10**6, acc_bef_sub_away_impl_min =-10**6, conf_1_sub_home_impl_min =-10**6, conf_0_sub_away_impl_min =-10**6, type_bet = "bet the spread!"  ): #or "bet_100"
    #
    ##return df with columns for each season in question:
    #type_bet,  total_invested, total_earned, profit, ROI, 
    ##rows should be: avg for season in question let's do x_16 for now
    ###filters for df ..
    
    HA_bet = df['model_pred'].copy()  #0 or 1
    fav_pred = df['fav_pred'].copy()  #0 or 1 ... need 4 terms one for each combo
    actual_res= df['actual_res'].copy()  
    
    model_not_conf = (0.5 - conf_mib <= df_bet_info['model_conf_1'])&(df_bet_info['model_conf_1'] <= 0.5 + conf_min).copy() #within conf_min >=0 of 0.5 means not confident
    model_is_conf = (~model_not_conf).copy()
    
    ##restrict the df:
    
    df_bet_info['conf_1_sub_home_impl'] >= 
    
    #think about the two cases, pred = 1, 0; if bet home it makes conf_1 above home_impl_proba by certain amount  ... if bet away it makes conf_0 above ... 
    
    ##bet_fav filter
    
    if bet_fav and ~bet_und:
        filt_bet  = (HA_bet == fav_pred).copy()
    if ~bet_fav and bet_und:
        filt_bet = (HA_bet != fav_pred).copy()
    else:
        filt_bet = (HA_bet == HA_bet).copy() ##no filter
    
    filt_bet_fav = (HA_bet == fav_pred)|(~bet_fav)  # if bet_fav = False, want this to be all True; if bet_fav = True, want to use this filter as is; do OR not bet_fav
    filt_bet_und = (HA_bet != fav_pred)|(~bet_und)  #same deal ... if bet_und = True, want this filter to take effect; if False, want it to be all True
    filt_fav_und = filt_bet_fav & filt_bet_und   ##not possible to do both stragies ... 
    


    
    conf_above_impl_proba_home_bet = (HA_bet*df_bet_info['conf_1_sub_home_impl'] >= HA_bet*conf_1_sub_home_impl_min).copy()    #if HA_bet =0, reduced to 0>=0 (all True)
    conf_above_impl_proba_away_bet = ((1-HA_bet)*df_bet_info['conf_0_sub_away_impl'] >=(1-HA_bet)*conf_0_sub_away_impl_min ).copy()  #; #if HA_bet =1, reduced to 0>=0 
    conf_above_impl_proba_home = (HA_bet*df_bet_info['conf_1_sub_home_impl'] >= HA_bet*HA_bet*conf_1_sub_home_impl_min).copy()    #if HA_bet =0, reduced to 0>=0 (all True)
    conf_above_impl_proba_away = ((1-HA_bet)*df_bet_info['conf_0_sub_away_impl'] >=(1-HA_bet)*conf_0_sub_away_impl_min ).copy()  #; #if HA_bet =1, reduced to 0>=0 
        
   
    filt = (model_is_conf)&(conf_above_impl_proba_home&conf_above_impl_proba_away)
    
    
    df= df_bet_info.loc[filt, 'model_pred'].copy() ##right HERE is where we filter the rows down ... default should do no filtering
    
    
    
    
    
    
    #cases:    
    #bet = 0
        ##away_odds <0 (ie df['fav_pred'] = 0) ---> bet -away_odds profit  100
       ##away_odds >0 (ie df['fav_pred'] = 1) ---> bet 100        profit away_odds
    
    bet_A_A = (1- bet_HA)*(1-fav_pred)*(-df['away_odds']).copy()
    win_A_A = ((1- bet_HA)*(1-fav_pred)(1-actual_res)*100).copy()   
        
    bet_A_H = ((1-bet_HA)*fav_pred*100).copy()  
    win_A_H = ((1-bet_HA)*fav_pred*df['away_odds']*actual_res).copy()

    ##bet = 1  (ie df['model_pred'] = 1)
        ##home_odds >0 (ie df['fav_pred'] = 0) ---> bet 100        profit home_odds
        ##home_odds <0 (ie df['fav_pred'] = 1) ---> bet -home_odds profit  100
        
    bet_H_A = (bet_HA*(1-fav_pred)*100).copy()
    win_H_A = (bet_HA*(1-fav_pred)*df['home_odds']).copy()  
    
    bet_H_H = (bet_HA*fav_pred*(-df['home_odds']).copy()  
    win_H_H = (bet_HA*fav_pred*100).copy()  
    
    
    df['potential_profit'] = win_H_H + win_H_A + win_A_H + win_A_A 
    
    #if bet and result are same then  (1+ actual_result - bet_HA)  = 1 
    df['actual_profit'] = (bet_HA*actual_result)
               
               (-df['bet_amount'])(1+ actual_result + bet_HA)    +   (df['potential_profit'])(1+ actual_result - bet_HA)  
               
    
    df['profit'] = df['pay_out'] - df['bet_amount']       
    df['cumul_profit'] = 0
    
    for i in df.index:
        df.loc[i, 'cumul_profit'] = df.loc[:i, 'profit'].sum()
    
    df['total_profit'] = df['profit'].sum()    
    df['total_bet'] = df['bet'].sum()    
    df['total_ROI'] = df['profit'].sum() /df['bet'].sum()   
    
    #df.drop(columns = ['ones', 'non_pred'], inplace = True)
    return df

    
    